# Proyecto ANOVA Tercer Parcial

## Dylan Hernández Rojas

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy import special

Importamos primero la información desde el csv:

In [2]:
csv_path = 'https://github.com/dylanhrojas/AnovaProjectInegi/raw/main/indice_de_remuneraciones_por_persona_por_subsector.csv'
raw_df = pd.read_csv(csv_path)

Se obtienen los índices de:
- Fabricación de maquinaria y equipo
- Fabricación de equipo de transporte
- Fabricación de muebles, colchones y persianas

Se toman del $[14,26)$ debido a que se toman los 12 índices del año 2023.

In [3]:
fabricacion_de_maquinaria_y_equipo = raw_df['Fabricacion de maquinaria y equipo']
fabricacion_de_maquinaria_y_equipo_2023 = fabricacion_de_maquinaria_y_equipo.iloc[14:26]
fabricacion_de_maquinaria_y_equipo_2023

14    5.1
15    1.3
16    2.7
17    1.7
18    6.1
19    4.5
20    5.4
21    8.6
22    1.9
23    3.1
24    6.0
25    3.0
Name: Fabricacion de maquinaria y equipo, dtype: float64

Por cada uno se realiza una muestra.

In [4]:
fabricacion_de_maquinaria_y_equipo_2023_sample = fabricacion_de_maquinaria_y_equipo_2023.sample(n=5, random_state=1, ignore_index=True)
fabricacion_de_maquinaria_y_equipo_2023_sample

0    2.7
1    1.7
2    6.1
3    6.0
4    1.3
Name: Fabricacion de maquinaria y equipo, dtype: float64

In [5]:
fabricacion_de_equipo_de_transporte = raw_df['Fabricación de equipo de transporte']
fabricacion_de_equipo_de_transporte_2023 = fabricacion_de_equipo_de_transporte.iloc[14:26]
fabricacion_de_equipo_de_transporte_2023 

14    2.1
15    2.7
16    6.7
17    3.0
18    7.6
19    6.4
20    3.5
21    6.5
22    3.8
23    6.8
24    8.9
25    3.6
Name: Fabricación de equipo de transporte, dtype: float64

In [6]:
fabricacion_de_equipo_de_transporte_2023_sample = fabricacion_de_equipo_de_transporte_2023.sample(n=4, random_state=1, ignore_index=True)
fabricacion_de_equipo_de_transporte_2023_sample 

0    6.7
1    3.0
2    7.6
3    8.9
Name: Fabricación de equipo de transporte, dtype: float64

In [7]:
fabricacion_de_muebles_colchones_y_persianas = raw_df['Fabricación de muebles, colchones y persianas']
fabricacion_de_muebles_colchones_y_persianas_2023 = fabricacion_de_muebles_colchones_y_persianas.iloc[14:26]
fabricacion_de_muebles_colchones_y_persianas_2023

14    11.4
15     8.0
16     9.7
17     8.6
18    12.4
19    10.9
20     7.5
21    13.8
22     5.4
23     9.3
24    12.3
25     7.8
Name: Fabricación de muebles, colchones y persianas, dtype: float64

In [8]:
fabricacion_de_muebles_colchones_y_persianas_2023_sample = fabricacion_de_muebles_colchones_y_persianas_2023.sample(n=5, random_state=1, ignore_index = True)
fabricacion_de_muebles_colchones_y_persianas_2023_sample

0     9.7
1     8.6
2    12.4
3    12.3
4     8.0
Name: Fabricación de muebles, colchones y persianas, dtype: float64

Se crea un DataFrame con la información de las muestras.

In [9]:
data = {
    'A': fabricacion_de_maquinaria_y_equipo_2023_sample,
    'B': fabricacion_de_equipo_de_transporte_2023_sample,
    'C': fabricacion_de_muebles_colchones_y_persianas_2023_sample
}

df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data.items()]))
df

,A,B,C
0,2.7,6.7,9.7
1,1.7,3.0,8.6
2,6.1,7.6,12.4
3,6.0,8.9,12.3
4,1.3,NaN,8.0


$H_0$: $\mu_A = \mu_B = \mu_C$

$H_a$: Por lo menos un índice es diferente 

$\alpha = 0.05$

In [10]:
grand_mean = df.stack().mean()
means = df.mean()
means

A     3.56
B     6.55
C    10.20
dtype: float64

In [11]:
k = len(data)
n = df.count().sum()

*Between Groups*

In [12]:
between_ss = sum(df.count()[group] * (mean - grand_mean)**2 for group, mean in means.items())
between_df = k - 1
between_ms = between_ss / between_df

*Within Groups*

In [13]:
within_ss = sum(((df[group] - means[group])**2).sum() for group in df)
within_df = n - k
within_ms = within_ss / within_df

*Valor de F y P*

In [14]:
f = between_ms / within_ms

In [15]:
p = 1 - stats.f.cdf(f, between_df, within_df)

*Total*

In [16]:
total_ss = between_ss + within_ss
total_df = n -1

**Creación de DataFrame ANOVA**

In [17]:
anova_df = pd.DataFrame(
    {
        'Source of variation': ['Between Groups', 'Within Groups', 'Total'],
        'SS': [between_ss, within_ss, total_ss],
        'df': [between_df, within_df, total_df],
        'MS': [between_ms, within_ms, 0],
        'F' : [f, 0, 0],
        'P' : [p, 0, 0]
    }
)
anova_df

,Source of variation,SS,df,MS,F,P
0,Between Groups,110.535143,2,55.267571,10.50678,0.002808
1,Within Groups,57.862000,11,5.260182,0.00000,0.000000
2,Total,168.397143,13,0.000000,0.00000,0.000000


En este caso tenemos que $\alpha$=0.05. Por lo tanto podemos decir que 0.002808 (valor de P obtenido) < 0.05.
En conclusión, se rechaza $H_0$

## Prueba LSD

In [18]:
t = abs(special.stdtrit(within_df, 0.025))
t

np.float64(2.200985160082949)

Valores LSD (en este caso hay tres)

In [19]:
lsd1 = t * np.sqrt(within_ms*(1/(df['A'].count()) + 1/(df['B'].count())))
lsd1

np.float64(3.3862868893875535)

In [20]:
lsd2 = t * np.sqrt(within_ms*(1/(df['B'].count()) + 1/(df['C'].count())))
lsd2

np.float64(3.3862868893875535)

In [21]:
lsd3 = t * np.sqrt(within_ms*(1/(df['A'].count()) + 1/(df['C'].count())))
lsd3

np.float64(3.192621896705386)

In [22]:
x1, x2, x3 = means.iloc[:3]

*Prueba 1*

$H_0$: $\mu_A = \mu_B$

$H_a$: $\mu_A$ $\neq$ $\mu_B$

$\bar{x}_1$

In [23]:
x1

3.56

$\bar{x}_2$

In [24]:
x2

6.549999999999999

*Regla de decisión*

In [25]:
abs(x1 - x2) > lsd1

np.False_

$\therefore$ no se rechaza $H_0$

Índices de remuneración $\mu_A$ y $\mu_B$ son iguales.

*Prueba 2*

$H_0$: $\mu_B = \mu_C$

$H_a$: $\mu_B$ $\neq$ $\mu_C$

$\bar{x}_2$

In [26]:
x2

6.549999999999999

$\bar{x}_3$

In [27]:
x3

10.2

In [28]:
abs(x2 - x3) > lsd2

np.True_

$\therefore$ se rechaza $H_0$

Índices de remuneración $\mu_B$ y $\mu_C$ no son iguales.

*Prueba 3*

$H_0$: $\mu_A = \mu_C$

$H_a$: $\mu_A$ $\neq$ $\mu_C$

$\bar{x}_1$

In [29]:
x1

3.56

$\bar{x}_3$

In [30]:
x3

10.2

In [31]:
abs(x1 - x3) > lsd3

np.True_

$\therefore$ se rechaza $H_0$

Índices de remuneración $\mu_A$ y $\mu_C$ no son iguales.